# Gotobi Strategy Backtest Demo

Demonstrates running the Gotobi FX settlement strategy using NautilusTrader's BacktestEngine.
Loads USDJPY data from parquet files and runs both the basic Gotobi and stop-loss variants.

In [1]:
from pathlib import Path
import sys
import importlib

import pandas as pd
import plotly.graph_objects as go

from nautilus_trader.backtest.engine import BacktestEngineConfig
from nautilus_trader.common.config import LoggingConfig
from nautilus_trader.model.data import BarType


def find_project_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if (candidate / "pyproject.toml").exists() and (candidate / "trader").exists():
            return candidate
    raise RuntimeError("Could not find project root containing pyproject.toml and trader/.")


PROJECT_ROOT = find_project_root(Path.cwd().resolve())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from trader import DataHandler, SIM

import trader.data.catalog as catalog
importlib.reload(catalog)
dataframe_to_nautilus_bars = catalog.dataframe_to_nautilus_bars

import trader.core.instruments as instruments
importlib.reload(instruments)
make_fx_pair = instruments.make_fx_pair

import trader.config.node as node
importlib.reload(node)
build_backtest_engine = node.build_backtest_engine

import trader.strategy.gotobi as gotobi
importlib.reload(gotobi)
GotobiStrategy = gotobi.GotobiStrategy
GotobiConfig = gotobi.GotobiConfig
GotobiWithSLStrategy = gotobi.GotobiWithSLStrategy
GotobiWithSLConfig = gotobi.GotobiWithSLConfig

print(f"Project root: {PROJECT_ROOT}")


Project root: /Users/justin/Algo-Trading


## Load Data

In [2]:
# Load USDJPY parquet data

data_candidates = [
    PROJECT_ROOT / "data" / "USDJPY_1min_2024-01-01_2025-10-01.parquet",
    PROJECT_ROOT / "data" / "usdjpy_1min_2024-01-01_2025-10-01.parquet",
]
data_path = next((path for path in data_candidates if path.exists()), None)
if data_path is None:
    raise FileNotFoundError(f"Could not find USDJPY parquet in: {data_candidates}")

handler = DataHandler()
df = handler.load_parquet(str(data_path), tz="Asia/Tokyo")

print(f"Loaded {len(df):,} bars from {df.index[0]} to {df.index[-1]}")
print(f"Data path: {data_path}")


Loaded 644,043 bars from 2024-01-01 09:32:00+09:00 to 2025-10-01 08:59:00+09:00
Data path: /Users/justin/Algo-Trading/data/USDJPY_1min_2024-01-01_2025-10-01.parquet


## Build Instruments and Bar Data

In [3]:
# Create USDJPY instrument
instrument = make_fx_pair("USDJPY", SIM, lot_size=100_000)

# Define bar type
bar_type = BarType.from_str(f"{instrument.id}-1-MINUTE-MID-EXTERNAL")

# Convert DataFrame to NautilusTrader bars
bars = dataframe_to_nautilus_bars(df, bar_type, price_precision=instrument.price_precision)
print(f"Converted {len(bars):,} bars for {bar_type}")


Converted 644,043 bars for USD/JPY.SIM-1-MINUTE-MID-EXTERNAL


## Run Gotobi Strategy (No Stop Loss)

In [19]:
# Configure strategies
long_config = GotobiConfig(
    instrument_id=str(instrument.id),
    bar_type=str(bar_type),
    entry_time="09:00:00",
    exit_time="09:55:00",
    trade_size=50.0,
    contract_size=1.0,
    use_holidays=True,
)

short_config = GotobiConfig(
    instrument_id=str(instrument.id),
    bar_type=str(bar_type),
    entry_time="10:00:00",
    exit_time="10:45:00",
    trade_size=-50.0,
    contract_size=1.0,
    use_holidays=True,
)

long_strat = GotobiStrategy(config=long_config)
short_strat = GotobiStrategy(config=short_config)

venue_currency = instrument.base_currency.code
starting_balance = 150_000


def build_engine(strategies):
    return build_backtest_engine(
        instruments=[instrument],
        bars={bar_type: bars},
        strategies=strategies,
        venue=SIM,
        venue_currency=venue_currency,
        starting_balance=starting_balance,
        leverage=500.0,
        config=BacktestEngineConfig(
            logging=LoggingConfig(log_level="ERROR", log_colors=False),
        ),
    )


# Build and run engine
engine = build_engine([long_strat, short_strat])
engine.run()
print(f"Backtest complete. Analysis currency: {venue_currency}")


Backtest complete. Analysis currency: USD


## Run Gotobi With Stop Loss

In [31]:
# Reset and run with stop-loss variant
long_sl_config = GotobiWithSLConfig(
    instrument_id=str(instrument.id),
    bar_type=str(bar_type),
    entry_time="09:00:00",
    exit_time="09:55:00",
    trade_size=50.0,
    contract_size=1.0,
    stop_loss_pct=0.001,
    use_holidays=True,
)

short_sl_config = GotobiWithSLConfig(
    instrument_id=str(instrument.id),
    bar_type=str(bar_type),
    entry_time="10:00:00",
    exit_time="10:45:00",
    trade_size=-50.0,
    contract_size=1.0,
    stop_loss_pct=0.001,
    use_holidays=True,
)

long_sl = GotobiWithSLStrategy(config=long_sl_config)
short_sl = GotobiWithSLStrategy(config=short_sl_config)

engine_sl = build_engine([long_sl, short_sl])
engine_sl.run()
print("Stop-loss backtest complete.")


Stop-loss backtest complete.


In [32]:
analysis_currency = instrument.base_currency
analysis_currency_code = analysis_currency.code


def analyze_engine(engine, label):
    analyzer = engine.portfolio.analyzer
    pnl_stats = analyzer.get_performance_stats_pnls(currency=analysis_currency)
    returns_stats = analyzer.get_performance_stats_returns()
    general_stats = analyzer.get_performance_stats_general()

    # Use per-trade return series to reconstruct an equity curve.
    returns = analyzer.returns().sort_index()
    equity_curve = (1.0 + returns).cumprod() * starting_balance
    equity_curve.name = label

    return pnl_stats, returns_stats, general_stats, equity_curve


stats_pnls_base, stats_returns_base, stats_general_base, equity_base = analyze_engine(engine, "Gotobi")
stats_pnls_sl, stats_returns_sl, stats_general_sl, equity_sl = analyze_engine(engine_sl, "Gotobi + Stop Loss")

stats_pnls = pd.DataFrame([stats_pnls_base, stats_pnls_sl], index=["Gotobi", "Gotobi + Stop Loss"])
stats_returns = pd.DataFrame([stats_returns_base, stats_returns_sl], index=["Gotobi", "Gotobi + Stop Loss"])
stats_general = pd.DataFrame([stats_general_base, stats_general_sl], index=["Gotobi", "Gotobi + Stop Loss"])

equity_curves = pd.concat([equity_base, equity_sl], axis=1).dropna(how="all")
if equity_curves.index.tz is not None:
    equity_curves.index = equity_curves.index.tz_convert("Asia/Tokyo")

fig = go.Figure()
for column in equity_curves.columns:
    fig.add_trace(
        go.Scatter(
            x=equity_curves.index,
            y=equity_curves[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title=f"Gotobi Equity Curves ({analysis_currency_code})",
    xaxis_title="Time (Asia/Tokyo)",
    yaxis_title=f"Equity ({analysis_currency_code})",
    template="plotly_white",
)

fig.show()


In [7]:
summary = pd.DataFrame(
    {
        "Final Equity": equity_curves.ffill().iloc[-1],
        "Total PnL": stats_pnls["PnL (total)"],
    }
)
summary


,Final Equity,Total PnL
Gotobi,152783.860209,180.0
Gotobi + Stop Loss,149514.244651,-25.0


In [8]:
equity_html_path = PROJECT_ROOT / "tests" / "notebooks" / "backtest_equity_curves.html"
fig.write_html(str(equity_html_path), include_plotlyjs="cdn")
print(f"Saved equity curve chart: {equity_html_path}")


Saved equity curve chart: /Users/justin/Algo-Trading/tests/notebooks/backtest_equity_curves.html


In [9]:
engine.get_result()


BacktestResult(trader_id='BACKTESTER-001', machine_id='Justins-MacBook-Pro.local', run_config_id=None, instance_id='34929709-1308-4108-871e-1c1ee9d985c3', run_id='356d9829-c301-4278-8cf5-1758ed63edfe', run_started=1770709465413897000, run_finished=1770709490191989000, backtest_start=1704069120000000000, backtest_end=1759276740000000000, elapsed_time=55207620.0, iterations=644043, total_events=120622, total_orders=60311, total_positions=45, stats_pnls={'JPY': {'PnL (total)': 180.0, 'PnL% (total)': 0.12, 'Max Winner': 82.0, 'Avg Winner': 2.9132882882882885, 'Min Winner': 1.0, 'Min Loser': 0.0, 'Avg Loser': -1.7989913544668588, 'Max Loser': -95.0, 'Expectancy': 0.03954305799648529, 'Win Rate': 0.39015817223198596}}, stats_returns={'Returns Volatility (252 days)': 0.07747840028954255, 'Average (Return)': 4.275777109894432e-06, 'Average Loss (Return)': -0.00030364082038292174, 'Average Win (Return)': 0.0003150713810797133, 'Sharpe Ratio (252 days)': 0.9096015596866638, 'Sortino Ratio (252 d

In [10]:
engine_sl.get_result()


BacktestResult(trader_id='BACKTESTER-001', machine_id='Justins-MacBook-Pro.local', run_config_id=None, instance_id='8dbe441d-a3eb-4fdd-8510-736717367f7d', run_id='8c49e34d-87d4-4d7c-bb53-b8c0d9bb01bb', run_started=1770709492443312000, run_finished=1770709522591752000, backtest_start=1704069120000000000, backtest_end=1759276740000000000, elapsed_time=55207620.0, iterations=644043, total_events=195502, total_orders=97720, total_positions=69, stats_pnls={'JPY': {'PnL (total)': -25.0, 'PnL% (total)': -0.016666666666666666, 'Max Winner': 82.0, 'Avg Winner': 2.80863989200135, 'Min Winner': 1.0, 'Min Loser': 0.0, 'Avg Loser': -1.8503657725559743, 'Max Loser': -95.0, 'Expectancy': -0.003344929087503303, 'Win Rate': 0.39644099545089645}}, stats_returns={'Returns Volatility (252 days)': 0.04981469967388077, 'Average (Return)': -1.4706007634868433e-06, 'Average Loss (Return)': -0.0003055881032989739, 'Average Win (Return)': 0.0003039022887793711, 'Sharpe Ratio (252 days)': -0.20639016444938924, '